In [1]:
import sys

my_list = [1, 2, 3]
print(sys.getrefcount(my_list))

another_ref = my_list
print(sys.getrefcount(my_list))

another_ref = None
print(sys.getrefcount(my_list))



2
3
2


In [2]:
import gc
import weakref

class Node:
    def __init__(self, value):
        self.value = value
        self.children = []
        self.parent = None

    def add_child(self, child):
        child.parent = self
        self.children.append(child)

parent = Node("parent")
child = Node("child")
parent.add_child(child)

print(f"Objects before: {len(gc.get_objects())}")

collected = gc.collect()
print(f"Objects collected: {collected}")

def callback(ref):
    print("Object was garbage collected!")


weak_ref = weakref.ref(parent, callback)
parent = None
child = None
gc.collect()

Objects before: 77526
Objects collected: 371
Object was garbage collected!


4

In [3]:
import gc
import sys
from collections import defaultdict

def memory_usage():
    stats = defaultdict(int)
    for obj in gc.get_objects():
        stats[type(obj).__name__] += 1
    return dict(stats)

def find_memory_leaks():
    gc.set_debug(gc.DEBUG_LEAK)

    data = []
    for i in range(1000):
        data.append([i] * 100)

    collected = gc.collect()
    if gc.garbage:
        print(f"Memory leaks detected: {len(gc.garbage)} objects")
        for obj in gc.garbage:
            print(f"Leaked object: {type(obj)}")

    return collected

print("Memory before:", memory_usage())
find_memory_leaks()
print("Memory after:", memory_usage())

Memory before: {'dict': 9378, 'IPythonDisplayFormatter': 1, 'list': 5363, 'MimeBundleFormatter': 1, 'PlainTextFormatter': 1, 'HTMLFormatter': 2, 'MarkdownFormatter': 1, 'SVGFormatter': 1, 'PNGFormatter': 1, 'PDFFormatter': 1, 'JPEGFormatter': 1, 'LatexFormatter': 1, 'JSONFormatter': 1, 'JavascriptFormatter': 1, 'cell': 2482, 'tuple': 10865, 'function': 18495, 'HistoryOutput': 3, 'defaultdict': 18, 'coroutine': 14, 'Future': 8, 'builtin_function_or_method': 1418, 'TimerHandle': 1, 'Context': 29, 'FutureIter': 3, 'Frame': 7, 'Handle': 11, 'method': 2401, 'partial': 14, 'Condition': 92, 'RLock': 16, 'deque': 52, 'hamt_bitmap_node': 3, 'hamt': 3, '_GeneratorContextManager': 5, 'generator': 5, 'ExecutionInfo': 2, 'ExecutionResult': 2, 'ReferenceType': 3795, 'Module': 2, 'Import': 2, 'alias': 3, 'ImportFrom': 1, 'FunctionDef': 2, 'arguments': 2, 'Assign': 3, 'Name': 34, 'Call': 17, 'For': 3, 'Attribute': 9, 'AugAssign': 1, 'Subscript': 1, 'Constant': 8, 'Return': 2, 'Expr': 7, 'List': 43, 'B

In [4]:
import weakref

class ExpensiveResource:
    def __init__(self, name):
        self.name = name
        print(f"Creating expensive resource: {name}")

    def __del__(self):
        print(f"Destroying resource: {self.name}")

resource = ExpensiveResource("database")
regular_ref = resource


weak_ref = weakref.ref(resource)

print(f"Weak reference alive: {weak_ref() is not None}")

resource = None
regular_ref = None

import gc
gc.collect()
print(f"Weak reference alive: {weak_ref() is not None}")

Creating expensive resource: database
Weak reference alive: True
Destroying resource: database
Weak reference alive: False


In [6]:
def demonstrate_exceptions():
    exceptions_demo = {
        'ValueError': lambda: int('not_a_number'),
        'TypeError': lambda: 'string' + 5,
        'indexError': lambda: [1, 2, 3][10],
        'KeyError': lambda: {'a': 1}['b'],
        'FileNotFoundError': lambda: open('nonexistent.txt'),
        'ZeroDivisionError': lambda: 1 / 0,
        'AttributeError': lambda: 'string'.noneistent_method()
    }
    for exc_name, func in exceptions_demo.items():
        try:
            func()
        except Exception as e:
            print(f"{exc_name}: {e}")

demonstrate_exceptions()

ValueError: invalid literal for int() with base 10: 'not_a_number'
TypeError: can only concatenate str (not "int") to str
indexError: list index out of range
KeyError: 'b'
FileNotFoundError: [Errno 2] No such file or directory: 'nonexistent.txt'
ZeroDivisionError: division by zero
AttributeError: 'str' object has no attribute 'noneistent_method'


In [7]:
def safe_division(a, b):
    try:
        result = a / b
        return result
    except ZeroDivisionError:
        print("Cannot divide by zero!")
        return None
    except TypeError:
        print("Invalid types for division!")
        return None

def process_data(data):
    try:
        result = int(data) * 2
        return result
    except (ValueError, TypeError) as e:
        print(f"Data processing error: {e}")
        return 0

def file_processor(filename):
    try:
        file = open(filename, 'r')
        data = file.read()
    except FileNotFoundError:
        print(f"File {filename} not found")
        return None
    except PermissionError:
        print(f"Permission denied for {filename}")
        return None
    else:
        print("File already successfully")
        return data
    finally:
        try:
            file.close()
            print("File closed")
        except:
            pass

result = safe_division(10, 2)
result = safe_division(10, 0)
result = process_data("abc")
        

Cannot divide by zero!
Data processing error: invalid literal for int() with base 10: 'abc'


In [8]:
def validate_age(age):
    if not isinstance(age, int):
        raise TypeError("Age must be an integer")

    if age < 0:
        raise ValueError("Age cannot be negative")

    if age > 150:
        raise ValueError("Age seems unrealistic")

    return age

def wrapper_function(age):
    try:
        return validate_age(age)
    except ValueError as e:
        print(f"Valdation error: {e}")
        raise

try:
    age = validate_age(-5)
except ValueError as e:
    print(f"Caught: {e}")

Caught: Age cannot be negative


In [4]:
class ValidationError(Exception):
    def __init__(self, message, code = None):
        super().__init__(message)
        self.code = code

class EmailValidationError(ValidationError):
    pass

class PasswordValidationError(ValidationError):
    pass

def validate_email(email):
    if '@' not in email:
        raise EmailValidationError("Email domain must contain @", code="MISSING_AT")

    if '.' not in email.split('@')[1]:
        raise EmailValidationError("Email domain must contain .", code = "MISSING_DOT")

    return email

def validate_password(password):
    if len(password) < 8:
        raise PasswordValidationError("Password must be atleast 8 characters", code="TOO_SHORT")
        
    if not any(c.isupper() for c in password):
        raise PasswordValidationError("Password must contain uppercase letter", code="NO_UPPER")

    return password

def register_user(email, password):
    try:
        validate_email(email)
        validate_password(password)

        return f"User registration with email: {email}"
    except EmailValidationError as e:
        return f"Email error ({e.code}): {e}"
    except PasswordValidationError as e:
        return f"Password error ({e.code}): {e}"

    except ValidationError as e:
        return f"General validation error: {e}"

print(register_user("invalid-email", "short"))
print(register_user("user@domain.com", "ValidPassword123"))

Email error (MISSING_AT): Email domain must contain @
User registration with email: user@domain.com


In [ ]:
import traceback
import sys

def detailed_exception_handling():
    try:
        data = {'users': [{'name': 'Alice', 'age': 30}]}
        user_age = data['users'][0]['nonexistent_key']

    except Exception as e:
        exc_type, exc_value, exc_traceback = sys.exc_info()

        print(f"Exeption type: {exc_type.__name__}")
        print(f"Exception value: {exc_value}")
        print(f"Exception args: {e.args}")

        print("\nTraceback:")
        traceback.print_exception(exc_type, exc_value, exc_traceback)
        print("\nTraceback as string:")
        print(''.join(tb_str))

detailed_exception_handling()

In [7]:
from typing import List, Dict, Tuple, Optional, Union, Any

def greet(name: str) -> str:
    return f"Hello. {name}!"

def calculate_age(birth_year: int) -> int:
    from datetime import datetime
    return datetime.now().year - birth_year

def process_scores(scores: List[int]) -> float:
    return sum(scores) / len(scores)

def get_user_info(user_id: int) -> Dict[str, Any]:
    return{
        'id': user_id,
        'name': 'John Doe',
        'age': 30,
        'active': True
    }

def get_coordinates() -> Tuple[float, float]:
    return (10.5, 20.6)

def find_user(user_id: int) -> Optional[Dict[str, Any]]:
    if user_id == 1:
        return {'id': 1, 'name': 'Alice'}
    return None

def process_id(user_id: Union[int, str]) -> str:
    return str(user_id)

print(greet("Alice"))
print(calculate_age(1990))
print(process_scores([85, 92, 78, 96]))

Hello. Alice!
35
87.75


In [ ]:
from typing import TypeVar, Generic, Protocol, Callable, Iterator

T = TypeVar('T')
K = TypeVar('K')
V = TypeVar('V')

class Stack(Generic[T]):
    def __init__(self) -> None:
        self._items: List[T] = []

    def push(self, item: T) -> None:
        self._items.append(item)

    def pop(self) -> T:
        return self._items.pop()

    def peek(self) -> T:
        return self._items[-1]

    def is_empty(self) -> bool:
        return len(self._items) == 0

class Drawable(Protocol):
    def draw(self) -> None: ...

class Circle:
    def draw(self) -> None:
        print("Drawing a circle")

class Square:
    def draw(self) -> None:
        print("Drawing a square")

def render_shape(shape: Drawable) -> None:
    shape.draw()


def apply_operation(numbers: List[int], operation: Callable[[int], int]) -> List[int]:
        return [operation(num) for num in numbers]

def fibonacci(n: int) -> Iterator[int]:
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b

int_stack = Stack[int]()
int_stack.push(42)
print(int_stack.pop())

string_stack = Stack[str]()
string_stack.push("hello")
print(string_stack.pop())

render_shape(Circle())
render_shape(Square())

doubled = apply_operation([1, 2, 3, 4], lambda x: x * 2)
print(doubled)


In [10]:
from typing import Literal, Final
from enum import Enum

def set_color(color: Literal['red', 'green', 'blue']) -> None:
    print(f"Setting color to {color}")

MAX_CONNECTIONS: Final = 100
API_VERSION: Final[str] = "v1.0"

class Config:
    DATABASE_URL: Final[str] = "postgresql://localhost/mydb"

from typing import TypedDict

class PersonDict(TypedDict):
    name: str
    age: int
    email: str

def create_person(data: PersonDict) -> PersonDict:
    return{
        'name': data['name'],
        'age': data['age'],
        'email': data['email']
        
    }

set_color('red')

    

Setting color to red


In [11]:
set_color('yellow')

Setting color to yellow


In [13]:
person_data: PersonDict = {
    'name': 'Alice',
    'age': 30,
    'email': 'alice@example.com'
}

In [15]:
from typing import TYPE_CHECKING
if TYPE_CHECKING:
    from expensive_module import ExpensiveClass

def process_data(data: 'ExpensiveClass') -> None:
    pass

def validate_type(value: Any, expected_type: type) -> bool:
    return isinstance(value, expected_type)

def is_string_list(value: Any) -> bool:
    return (isinstance(value, list) and
           all(isinstance(item, str) for item in value))

def process_strings(data: Any) -> None:
    if is_string_list(data):
        for item in data:
            print(item.upper())

In [16]:
class Person:
    species = "Homo sapiens"

    def __init__(self, name, age):
        self.name = name
        self.age = age

    def greet(self):
        return f"Hello, i'm {self.name}"

person = Person("Alice", 30)

print("All Attributes:", dir(person))

print("Has name:", hasattr(person, 'name'))
print("Has height:", hasattr(person, 'height'))

print("Name:", getattr(person, 'name'))
print("Height:", getattr(person, 'height', 'Unknown'))

setattr(person, 'height', 170)
print("Height after setting:", person.height)

delattr(person, 'height')
print("Has height after deletion:", hasattr(person, 'height'))

print("Instance dict:", person.__dict__)
print("Class dict:", person.__dict__)

All Attributes: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__firstlineno__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__static_attributes__', '__str__', '__subclasshook__', '__weakref__', 'age', 'greet', 'name', 'species']
Has name: True
Has height: False
Name: Alice
Height: Unknown
Height after setting: 170
Has height after deletion: False
Instance dict: {'name': 'Alice', 'age': 30}
Class dict: {'name': 'Alice', 'age': 30}


In [1]:
import inspect

def example_function(a: int, b: str = "default", *args, **kwargs) -> str:
    return f"a = {a}, b={b}, args={args}, kwargs={kwargs}"

sig = inspect.signature(example_function)
print("Signature:", sig)

for name, param in sig.parameters.items():
    print(f"Parameter: {name}")
    print(f" Default: {param.default}")
    print(f" Annotation: {param.annotation}")
    print(f" kind: {param.kind}")

print("Function name:", example_function.__name__)
print("Function doc:", example_function.__doc__)
print("Function module:", example_function.__module__)
print("Function annotation:", example_function.__annotations__)

print("Source code:")
print(inspect.getsource(example_function))

print("Is callable:", callable(example_function))

            

Signature: (a: int, b: str = 'default', *args, **kwargs) -> str
Parameter: a
 Default: <class 'inspect._empty'>
 Annotation: <class 'int'>
 kind: POSITIONAL_OR_KEYWORD
Parameter: b
 Default: default
 Annotation: <class 'str'>
 kind: POSITIONAL_OR_KEYWORD
Parameter: args
 Default: <class 'inspect._empty'>
 Annotation: <class 'inspect._empty'>
 kind: VAR_POSITIONAL
Parameter: kwargs
 Default: <class 'inspect._empty'>
 Annotation: <class 'inspect._empty'>
 kind: VAR_KEYWORD
Function name: example_function
Function doc: None
Function module: __main__
Function annotation: {'a': <class 'int'>, 'b': <class 'str'>, 'return': <class 'str'>}
Source code:
def example_function(a: int, b: str = "default", *args, **kwargs) -> str:
    return f"a = {a}, b={b}, args={args}, kwargs={kwargs}"

Is callable: True


In [4]:
square = lambda x: x ** 2
add = lambda a, b: a + b
filter_even = lambda ist: [x for x in ist if x % 2 == 0]

multiply = lambda x, y, z =1: x * y * z

numbers = [1, 2, 3, 4, 5]
squared = list(map(lambda x: x ** 2, numbers))
evens = list(filter(lambda x: x % 2 == 0, numbers))

students = [('Alice', 85), ('Bob', 92), ('Charlie', 78)]
sorted_by_grade = sorted(students, key=lambda student: student[1])
print("Sorted by grade:", sorted_by_grade)

max_value = lambda a, b: a if a > b else b
print("Max of 5 and 3:", max_value(5, 3))

print("Lambda name:", square.__name__)
print("Lambda code:", square.__code__.co_code)
print("Lambda args:", square.__code__.co_varnames)


Sorted by grade: [('Charlie', 78), ('Alice', 85), ('Bob', 92)]
Max of 5 and 3: 5
Lambda name: <lambda>
Lambda code: b'\x95\x00U\x00S\x01-\x08\x00\x00$\x00'
Lambda args: ('x',)


In [5]:
# Closures

def create_multiplier(factor):
    def multiply(x):
        return x * factor
    return multiply

double = create_multiplier(2)
triple = create_multiplier(3)

print("Double 5:", double(5))
print("Triple 5:", triple(5))

print("Closure variables:", double.__closure__)
print("Closure values:", [cell.cell_contents for cell in double.__closure__])


def create_counter():
    count = 0

    def increment():
        nonlocal count
        count += 1
        return count

    def get_count():
        return count

    return increment, get_count

increment, get_count = create_counter()

print("Count:", get_count())
print("After increment:", increment())
print("After increment:", increment())

        

Double 5: 10
Triple 5: 15
Closure variables: (<cell at 0x000001CF9B9AE980: int object at 0x00007FFAB62E73C8>,)
Closure values: [2]
Count: 0
After increment: 1
After increment: 2


In [6]:
class Vector:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __add__(self, other):
        return Vector(self.x + other.x, self.y + other.y)

    def __sub__(self, other):
        return Vector(self.x - other.x, self.y - other.y)

    def __mul__(self, scalar):
        return Vector(self.x / scalar, self.y / scalar)

    def __truediv__(self, scalar):
        return Vector(self.x / scalar, self.y / scalar)

    def __eq__(self, other):
        return self.x == other.x and self.y == other.y

    def __str__(self):
        return f"Vector({self.x}, {self.y})"

v1 = Vector(2, 3)
v2 = Vector(1, 4)

print("v1 + v2:", v1 + v2)
print("v1 - v2:", v1 - v2)
print("v1 * 2:", v1 * 2)
print("v1 / 2:", v1 / 2)
print("v1 == v2:", v1 == v2)

v1 + v2: Vector(3, 7)
v1 - v2: Vector(1, -1)
v1 * 2: Vector(1.0, 1.5)
v1 / 2: Vector(1.0, 1.5)
v1 == v2: False


In [7]:
class TimedOPeration:
    def __init__(self, operation_name):
        self.operation_name = operation_name
        self.start_time = None

    def __enter__(self):
        import time
        self.start_time = time.time()
        print(f"Starting {self.operation_name}")
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        import time
        end_time = time.time()
        duration = end_time - self.start_time
        print(f"{self.operation_name} took {duration:.2f} seconds")

        if exc_type is not None:
            print(f"Exception occured: {exc_val}")

        return False

with TimedOPeration("Data Processing") as timer:
    import time
    time.sleep(1)
    result = sum(range(1000000))

from contextlib import contextmanager

@contextmanager
def temporary_value(obj, attr, new_value):
    old_value = getattr(obj, attr)
    setattr(obj, attr, new_value)
    try:
        yield old_value
    finally:
        setattr(obj, attr, old_value)

class Config:
    debug = False

config = Config()
print("Debug before:", config.debug)

with temporary_value(config, 'debug', True):
    print("Debug during:", config.debug)

print("Debug after:", config.debug)  


Starting Data Processing
Data Processing took 1.02 seconds
Debug before: False
Debug during: True
Debug after: False


In [10]:
def fibonacci_generator(n):
    a, b = 0, 1
    for _ in range(n):
        yield a
        a, b = b, a + b

squares = (x ** 2 for x in range(10))

class CountDown:
    def __init__(self, start):
        self.start = start

    def __iter__(self):
        return self

    def __next__(self):
        if self.start <= 0:
            raise StopIteration
        self.start -= 1
        return self.start + 1

fib = fibonacci_generator(10)
print("Fibonacci:", list(fib))

print("Squares:", list(squares))

countdown = CountDown(5)
for num in countdown:
    print(f"Countdown: {num}")

Fibonacci: [0, 1, 1, 2, 3, 5, 8, 13, 21, 34]
Squares: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
Countdown: 5
Countdown: 4
Countdown: 3
Countdown: 2
Countdown: 1


In [13]:
import gc
import sys
from typing import Iterator, List, Any

class MemoryEfficientProcessor:
    def __init__(self, batch_size: int = 1000):
        self.batch_size = batch_size
        self.processed_count = 0

    def process_large_dataset(self, data: Iterator[Any]) -> Iterator[Any]:
        batch = []

        for item in data:
            batch.append(self.transform_item(item))

            if len(batch) >= self.batch_size:
                yield from self.process_batch(batch)
                batch = []

                if self.processed_count % (self.batch_size * 10) == 0:
                    collected = gc.collect()
                    print(f"Garbage collected: {collected} objects")

        if batch:
            yield from self.process_batch(batch)

    def transform_item(self, item: Any) -> Any:
        return item * 2 if isinstance(item, (int, float)) else str(item).upper()

    def process_batch(self, batch: List[Any]) -> Iterator[Any]:
        self.processed_count += len(batch)
        return iter(batch)

def data_generator():
    for i in range(100000):
        yield i

processor = MemoryEfficientProcessor(batch_size=1000)
processed_data = processor.process_large_dataset(data_generator())

for i, item in enumerate(processed_data):
    if i < 10:
        print(f"Processed item {i}: {item}")
    else:
        break


        
            

Processed item 0: 0
Processed item 1: 2
Processed item 2: 4
Processed item 3: 6
Processed item 4: 8
Processed item 5: 10
Processed item 6: 12
Processed item 7: 14
Processed item 8: 16
Processed item 9: 18


In [17]:
# Robust error handling

import logging
import traceback
from typing import Optional, Any, Callable
from functools import wraps

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ErrorHandler:
    def __init__(self, max_retries: int = 3):
        self.max_retries = max_retries
        self.error_count = 0
    
    def handle_with_retry(self, func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            for attempt in range(self.max_retries):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    self.error_count += 1
                    logger.warning(f"Attempt {attempt + 1} failed: {e}")
                    
                    if attempt == self.max_retries - 1:
                        logger.error(f"All {self.max_retries} attempts failed")
                        raise
                    
                    # Exponential backoff
                    import time
                    time.sleep(2 ** attempt)
            
            return None
        return wrapper

# Custom exception hierarchy
class DataProcessingError(Exception):
    """Base exception for data processing errors"""
    pass

class DataValidationError(DataProcessingError):
    """Raised when data validation fails"""
    pass

class DataTransformationError(DataProcessingError):
    """Raised when data transformation fails"""
    pass

class DataProcessor:
    def __init__(self):
        self.error_handler = ErrorHandler()
    
    @property
    def process_data(self):
        return self.error_handler.handle_with_retry(self._process_data)
    
    def _process_data(self, data: Any) -> Any:
        """Process data with comprehensive error handling"""
        try:
            # Validate data
            if not data:
                raise DataValidationError("Data cannot be empty")
            
            # Transform data
            if isinstance(data, str):
                if data.isdigit():
                    return int(data) * 2
                else:
                    raise DataTransformationError(f"Cannot transform non-numeric string: {data}")
            
            elif isinstance(data, (int, float)):
                return data * 2
            
            else:
                raise DataTransformationError(f"Unsupported data type: {type(data)}")
        
        except DataProcessingError:
            # Re-raise our custom exceptions
            raise
        except Exception as e:
            # Wrap unexpected exceptions
            raise DataProcessingError(f"Unexpected error: {e}") from e

# Usage
processor = DataProcessor()

test_data = ["10", "abc", 25, None, ""]

for data in test_data:
    try:
        result = processor.process_data(data)
        print(f"Processed {data}: {result}")
    except DataProcessingError as e:
        print(f"Processing error for {data}: {e}")


Processed 10: 20


ERROR:__main__:All 3 attempts failed


Processing error for abc: Cannot transform non-numeric string: abc
Processed 25: 50


ERROR:__main__:All 3 attempts failed


Processing error for None: Data cannot be empty


ERROR:__main__:All 3 attempts failed


Processing error for : Data cannot be empty


In [26]:
from typing import TypeVar, Generic, Dict, Any, Optional, Type
from dataclasses import dataclass
import json

T = TypeVar('T')

class ConfigValue(Generic[T]):
    def __init__(self, default: T, validator: Optional[Callable[[T], bool]] = None):
        self.default = default
        self.validator = validator
        self._value: Optional[T] = None

    def __get__(self, instance: Any, owner: Type) -> T:
        if instance is None:
            return self
        return self._value if self._value is not None else self.default

    def __set__(self, instance: Any, value: T) -> None:
        if self.validator and not self.validator(value):
            raise ValueError(f"Invalid value: {value}")
        self._value = value

@dataclass
class DatabaseConfig:
    host: str = "localhost"
    port: int = 5432
    database: str = "myapp"
    username: str = "user"
    password: str = "password"

class AppConfig:
    debug = ConfigValue(False, lambda x: isinstance(x, bool))
    max_connections = ConfigValue(100, lambda x: isinstance(x, int) and x > 0)
    api_key = ConfigValue("", lambda x: isinstance(x, str) and len(x) > 0)

    def __init__(self):
        self.database = DatabaseConfig()

    def load_from_file(self, filename: str) -> None:
        try:
            with open(filename, 'r') as f:
                config_data = json.load(f)

            for key, value in config_data.items():
                if key == 'database':
                    for db_key, db_value in value.items():
                        if hasattr(self.database, db_key):
                            setattr(self.database, db_key, db_value)
                else:
                    if hasattr(self, key):
                        setattr(self, key, value)

        except FileNotFoundError:
            logger.warning(f"Config file {filename} not found, using defaults")
        except json.JSONDecodeError as e:
            raise ValueError(f"Invalid Json in config file: {e}")


    def validate(self) -> bool:
        try:
            _ = self.debug
            _ = self.max_connections
            _ = self.api_key
            return True
        except ValueError as e:
            logger.error(f"Configuration validation failed: {e}")
            return False

config = AppConfig()

config.debug = True
config.max_connections = 200
config.api_key = "secret_key_123"

config.database.host = "production.db.com"
config.database.port = 5432

if config.validate():
    print("Configuration is valid")
    print(f"Debug mode: {config.debug}")
    print(f"Max connections: {config.max_connections}")
    print(f"Database: {config.database.host}:{config.database.port}")
else:
    print("Configuration validation failed")
        
            
        

Configuration is valid
Debug mode: True
Max connections: 200
Database: production.db.com:5432
